In [ ]:
!python --version

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# import du dataset
import pandas as pd
import seaborn as sns
import numpy as np
df = pd.read_csv('../dataset/edible_mushrooms.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(18, 8))
ax = sns.countplot(data = df, x = 'order', alpha = 0.7)

plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
ax.set_xlabel('order')
ax.set_ylabel('nombre de photos')
ax.set_title('nombre de photos par species pour les champignons comestibles');

# Balacing data

In [ ]:
# sous échantillonnage de 'df_species_uncom'
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

# Create the RandomUnderSampler object
undersample = RandomUnderSampler(sampling_strategy = 'auto')

# Undersample the dataframe
X = df.drop(columns = 'order')
y = df['order']
X_under, y_under = undersample.fit_resample(X, y)

In [ ]:
y_under.value_counts()

# Generating data

In [ ]:
path='../dataset/class_order'
train_path='../dataset/class_order/train'
test_path='../dataset/class_order/test'

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2,
    rotation_range = 45,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    vertical_flip = True,
    fill_mode = 'nearest')

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    directory = train_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [ ]:
test_generator = test_data_generator.flow_from_directory(
    directory = test_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

## balancing

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Get the true labels from the generator
y_train = train_generator.classes

# Compute the class weights
class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y_train), y = y_train)

# Create a dictionary mapping class indices to class weights
class_weight_dict = dict(enumerate(class_weights))


# VGG16 transfer learning

In [ ]:

import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
n_class= df['order'].unique().shape[0]

#base_model = MobileNet(weights = 'imagenet', include_top = False, input_shape=(224,224,3))
base_model = VGG16(weights = 'imagenet', include_top = False)

for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D()) 
model.add(Dense(1024,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(n_class, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', 
               loss='sparse_categorical_crossentropy', 
               metrics=['acc'])
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# Train the model on the preprocessed images and labels from the dataframe
batch_size = 16

#tf.debugging.disable_traceback_filtering()

model_history = model.fit_generator(
    train_generator,
    epochs = 5,
    validation_data=test_generator
)

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_history.history['acc'])
plt.plot(model_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# transfer learning on Mobile Net

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

n_class= df['order'].unique().shape[0]

base_model = MobileNet(weights = 'imagenet', include_top = False, input_shape=(224,224,3))

for layer in base_model.layers:
    layer.trainable = False
    
model_mobile = Sequential()
model_mobile.add(base_model)
model_mobile.add(GlobalAveragePooling2D()) 
model_mobile.add(Dense(1024,activation='relu'))
model_mobile.add(Dropout(rate=0.1))
model_mobile.add(Dense(n_class, activation='softmax'))

In [ ]:
model_mobile.compile(optimizer='adam', 
               loss='sparse_categorical_crossentropy', 
               metrics=['acc'])
model_mobile.summary()

In [ ]:
from keras.callbacks import EarlyStopping

# Create an early stopping callback
early_stopping = EarlyStopping(monitor='acc', patience=2, min_delta = 0.01)

model_mobile_history = model_mobile.fit(
    train_generator,
    epochs = 1,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_mobile_history.history['acc'])
plt.plot(model_mobile_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_mobile_history.history['acc'])
plt.plot(model_mobile_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
sns.heatmap(cm, annot=True)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
# we can observe the tendency to classify any mushroom as part of the dominant class, a consequence of imbalancing

## Individual image analysis for the model

In [ ]:
import torch
from torch import nn
from torchvision import models

base_model = models.mobilenet_v2(pretrained=True)
for param in base_model.parameters():
    param.requires_grad = False

model_mobile = nn.Sequential(
    base_model,
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(1280, 1024),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(1024, n_class),
    nn.Softmax(dim=1)
)

In [ ]:
import torch
from torch import nn, optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mobile.parameters())

print(model_mobile)

In [ ]:
model_mobile

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image
from torchvision.models import resnet50
from requests import get
import cv2

#model = resnet50(pretrained=True)
model = model_mobile

image_url = "https://th.bing.com/th/id/R.94b33a074b9ceeb27b1c7fba0f66db74?rik=wN27mvigyFlXGg&riu=http%3a%2f%2fimages5.fanpop.com%2fimage%2fphotos%2f31400000%2fBear-Wallpaper-bears-31446777-1600-1200.jpg&ehk=oD0JPpRVTZZ6yizZtGQtnsBGK2pAap2xv3sU3A4bIMc%3d&risl=&pid=ImgRaw&r=0"
img_stream = get(image_url, stream=True).raw
img = np.array(Image.open(img_stream))
img = cv2.resize(img, (224, 224))
img = np.float32(img) / 255
input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# Note: input_tensor can be a batch tensor with several images!


# The target for the CAM is the Bear category.
# As usual for classication, the target is the logit output
# before softmax, for that category.
targets = [ClassifierOutputTarget(295)]
#target_layers = [model.layer4]
target_layers = [model.layer4]

with GradCAM(model=model, target_layers=target_layers) as cam:
    grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
    cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)
cam = np.uint8(255*grayscale_cams[0, :])
cam = cv2.merge([cam, cam, cam])
images = np.hstack((np.uint8(255*img), cam , cam_image))
Image.fromarray(images)

## on the balanced dataset

In [ ]:
from keras.callbacks import EarlyStopping

# Create an early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Fit the model using the class weights
model_mobile_balanced_history = model_mobile.fit_generator(
    train_generator,
    class_weight=class_weight_dict,
    epochs = 10,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

# Assume `history` is the history object returned by the `fit` method
# Plot the accuracy and validation accuracy
plt.plot(model_mobile_balanced_history.history['acc'])
plt.plot(model_mobile_balanced_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

from sklearn.metrics import confusion_matrix

y_pred_proba = model_mobile.predict(test_generator)
y_pred = np.argmax(y_pred_proba, axis=1)

y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
sns.heatmap(cm, annot=True)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

# Background removal

In [ ]:
import imageio

# Read the image into a NumPy array
image = imageio.v3.imread('../dataset/images/0/18.jpg')

In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry["default"](checkpoint="../sam_vit_h_4b8939.pth")
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(image)

In [ ]:
import supervision as sv

mask_annotator = sv.MaskAnnotator()
detections = sv.Detections.from_sam(masks)
annotated_image = mask_annotator.annotate(image, detections)

In [ ]:
plt.figure(figsize=(10, 10))

# Display the image
plt.imshow(annotated_image)


# Remove the axis
plt.axis('off')

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))

# Display the image
plt.imshow(image)


# Remove the axis
plt.axis('off')

# Show the plot
plt.show()

In [ ]:
# derive background mask
background_mask = np.logical_not(np.logical_or.reduce(detections.mask))

# Apply the mask to the image
masked_image = np.where(background_mask[..., None], 0, image)

# Display image af
plt.figure(figsize=(10, 10))
plt.imshow(masked_image)

plt.axis('off')
plt.show()